# Step 1: Comprehensive Feature Extraction & Dataset Assembly

This notebook consolidates **all feature‑engineering steps** used in the project:

1. Physicochemical Properties (**PCP**)
2. Amino‑Acid Composition (**AAC**)
3. Network Centrality Metrics (from **Cytoscape**)
4. Gene‑Ontology (**GO**) one‑hot features (61 curated terms)

At the end, we **merge** every feature block into a single model‑ready CSV.

---

## Imports & Common Paths

In [ ]:
import pandas as pd, os, subprocess, json, itertools, numpy as np

# Adjust relative paths as needed for your repo structure
RAW_DIR = "./data/raw/"            # FASTA sequences
FEATURE_DIR = "./data/features/"    # Where feature blocks will be written
MERGED_OUT = "./data/processed/merged_dataset.csv"

os.makedirs(FEATURE_DIR, exist_ok=True)

## 1️⃣ PCP & AAC Extraction via *pfeature*

**Approach used in this project**  
- Leverage the standalone *pfeature* script (`pfeature_comp.py`) to batch‑process FASTA files.  
- For ≈20 k negative sequences, an **automated loop** was necessary; running each sequence interactively was infeasible.

Below is a *single* function that can be pointed at any FASTA file or folder and will output a CSV of either PCP or AAC features.

In [ ]:
def run_pfeature_batch(fasta_path: str, out_csv: str, job: str = "PCP"):
    """Run pfeature on a FASTA file and dump selected feature block.

    Parameters
    ----------
    fasta_path : str
        Path to input FASTA.
    out_csv : str
        Path to write CSV features.
    job : str
        'PCP' or 'AAC'.
    """
    script = "pfeature_comp.py"  # assuming it's on your PATH; change if necessary
    cmd = f"python {script} -i {fasta_path} -o {out_csv} -j {job}"
    subprocess.run(cmd, shell=True, check=True)

# Example (commented):
# run_pfeature_batch("./data/raw/positive_data_sample.fasta", f"{FEATURE_DIR}/pcp_pos.csv", "PCP")

> **Note** The full dataset extraction was executed offline prior to publishing to keep the repo lightweight.  
Re‑run the function above locally if you need to regenerate `pcp_*.csv` and `aac_*.csv`.

## 2️⃣ Network Centrality Metrics (Cytoscape)

1. Export *UniProt* interaction tables (with `Interacts with` column) for positive & negative sets.  
2. Import into **Cytoscape** → `Tools ▸ NetworkAnalyzer ▸ Analyze Network` (undirected).  
3. *NetworkAnalyzer* outputs a node table; export as TSV → drop it in `./data/features/centrality.tsv`.

The cell below simply loads that export and does minimal cleaning.

In [ ]:
centrality_path = f"{FEATURE_DIR}/centrality.tsv"  # ensure file placed manually
if os.path.exists(centrality_path):
    centrality_df = pd.read_csv(centrality_path, sep='\t')
    # keep a subset of informative columns
    keep_cols = [c for c in centrality_df.columns if c.lower() in {
        'degree', 'betweennesscentrality', 'closenesscentrality',
        'averageshortestpathlength', 'stress'
    }]
    centrality_df = centrality_df[['name'] + keep_cols]
else:
    centrality_df = pd.DataFrame()
centrality_df.head()

## 3️⃣ Gene Ontology (GO) Feature Engineering

###  Objective:
To identify GO terms **consistently enriched** in COVID-associated proteins compared to non-COVID (human) proteins,
and generate binary (one-hot) GO feature vectors for machine learning.

---

###  Process Summary:

1. **Input Files**:
   - `ppsitive_raw.csv`: COVID-positive proteins with GO annotations.
   - `uniprotkb_human_AND_reviewed_true_AND_m_2025_06_06.xlsx`: reviewed human proteins as negative examples.

2. **GO Parsing**:
   - Extracted `Gene Ontology IDs` from both datasets.
   - Built a **binary matrix**: 1 if protein has GO term, else 0.

3. **Statistical Testing**:
   - Ran **50 bootstraps**, each sampling negative proteins equal in number to positive ones.
   - For each GO term, applied **Fisher’s Exact Test**:
     > Is the term more common in COVID proteins than non-COVID?

4. **Multiple Testing Correction**:
   - Adjusted p-values using **FDR (Benjamini–Hochberg)**.
   - Counted how often each term was significant across all bootstraps.

5. **Thresholding**:
   - Selected GO terms significant in ≥70% of bootstraps.

6. **Output Files**:
   - `robust_covid_enriched_go_terms.csv`: GO terms enriched across bootstraps
   - `covid_go_features_filtered.csv`: One-hot GO matrix (COVID)
   - `noncovid_go_features_filtered.csv`: One-hot GO matrix (non-COVID)

These filtered GO features were then merged with PCP, AAC, and centrality features to build the final dataset.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
from collections import defaultdict

# === File paths ===
covid_file = './data/raw/ppsitive_raw.csv'
noncovid_file = './data/raw/uniprotkb_human_AND_reviewed_true_AND_m_2025_06_06.xlsx'

bootstrap_iters = 50
alpha = 0.05
min_consistency = 0.7

# === Load data ===
covid_df = pd.read_csv(covid_file)
noncovid_df = pd.read_excel(noncovid_file)

covid_df['GO_list'] = covid_df['Gene Ontology IDs'].fillna('').apply(lambda x: x.split(';') if x else [])
noncovid_df['GO_list'] = noncovid_df['Gene Ontology IDs'].fillna('').apply(lambda x: x.split(';') if x else [])

covid_size = covid_df.shape[0]
all_go_terms = set(term for row in pd.concat([covid_df['GO_list'], noncovid_df['GO_list']]) for term in row)
all_go_terms = sorted(all_go_terms)

def create_binary_matrix(df, go_terms):
    return pd.DataFrame(
        [[1 if go in terms else 0 for go in go_terms] for terms in df['GO_list']],
        columns=go_terms,
        index=df['Entry']
    )

covid_matrix = create_binary_matrix(covid_df, all_go_terms)
go_significance_counts = defaultdict(int)

for i in range(bootstrap_iters):
    noncovid_sample = noncovid_df.sample(n=covid_size, random_state=i).reset_index(drop=True)
    noncovid_matrix = create_binary_matrix(noncovid_sample, all_go_terms)

    p_values = []
    go_terms_tested = []

    for go in all_go_terms:
        a = covid_matrix[go].sum()
        c = noncovid_matrix[go].sum()
        b = covid_size - a
        d = covid_size - c
        if a + c == 0:
            continue
        _, p = fisher_exact([[a, b], [c, d]], alternative='greater')
        p_values.append(p)
        go_terms_tested.append(go)

    reject, _, _, _ = multipletests(p_values, alpha=alpha, method='fdr_bh')
    for go_term, is_sig in zip(go_terms_tested, reject):
        if is_sig:
            go_significance_counts[go_term] += 1

go_frac_significant = {go: count / bootstrap_iters for go, count in go_significance_counts.items()}
robust_go_terms = [go for go, frac in go_frac_significant.items() if frac >= min_consistency]

covid_filtered_matrix = covid_matrix[robust_go_terms]
noncovid_full_matrix = create_binary_matrix(noncovid_df, all_go_terms)[robust_go_terms]

covid_filtered_matrix.to_csv('./data/features/covid_go_features_filtered.csv')
noncovid_full_matrix.to_csv('./data/features/noncovid_go_features_filtered.csv')
pd.DataFrame(robust_go_terms, columns=['GO_term']).to_csv('./data/features/robust_covid_enriched_go_terms.csv', index=False)

print("Saved filtered GO feature matrices and robust term list.")

In [ ]:
import pandas as pd

# Load datasets
covid_df = pd.read_csv('./data/raw/ppsitive_raw.csv')
noncovid_df = pd.read_excel('./data/raw/uniprotkb_human_AND_reviewed_true_AND_m_2025_06_06.xlsx')

# Load curated GO terms
curated_go = pd.read_csv('./data/features/robust_covid_enriched_go_terms.csv')['GO_term'].tolist()

# Clean and split GO annotations
covid_df['GO_list'] = covid_df['Gene Ontology IDs'].fillna('').apply(lambda x: x.split(';') if x else [])
noncovid_df['GO_list'] = noncovid_df['Gene Ontology IDs'].fillna('').apply(lambda x: x.split(';') if x else [])

def make_one_hot(df, curated_terms):
    one_hot = pd.DataFrame(index=df['Entry'])
    for go in curated_terms:
        one_hot[go] = df['GO_list'].apply(lambda x: 1 if go in x else 0)
    return one_hot.reset_index()

# Create feature matrices
covid_go_features = make_one_hot(covid_df, curated_go)
noncovid_go_features = make_one_hot(noncovid_df, curated_go)

# Save
covid_go_features.to_csv('./data/features/covid_go_features_filtered.csv', index=False)
noncovid_go_features.to_csv('./data/features/noncovid_go_features_filtered.csv', index=False)

print("One-hot GO feature matrices saved:")
print("- covid_go_features_filtered.csv")
print("- noncovid_go_features_filtered.csv")


## 4️⃣ Merge Feature Blocks into One DataFrame

In [ ]:
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------------
# 1. Paths
# ------------------------------------------------------------------
FEATURE_DIR = Path("./data/features")
OUTPUT_FILE = Path("./data/processed/final_feature_matrix.csv")
OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------
# 2. Helper: read + rename the key column to a common name (“protein”)
# ------------------------------------------------------------------
def load_features(path, key_cols=("Entry", "name", "protein")):
    """Load a feature CSV/TSV and rename its key column to 'protein'."""
    df = pd.read_csv(path)
    for col in key_cols:
        if col in df.columns:
            df = df.rename(columns={col: "protein"})
            break
    return df

# ------------------------------------------------------------------
# 3. Load each feature block
#    – adjust filenames if yours differ
# ------------------------------------------------------------------
pcp_df        = load_features(FEATURE_DIR / "pcp_features.csv")
aac_df        = load_features(FEATURE_DIR / "aac_features.csv")
centrality_df = load_features(FEATURE_DIR / "centrality_features.csv")
go_df         = load_features(FEATURE_DIR / "covid_go_features_filtered.csv")
#  ⬆️  swap the GO file for the non‑COVID matrix when merging that set

# ------------------------------------------------------------------
# 4. Guard against duplicate NON‑key columns before merging
# ------------------------------------------------------------------
def drop_duplicate_cols(base, new):
    dupes = (set(base.columns) & set(new.columns)) - {"protein"}
    return new.drop(columns=list(dupes))

aac_df        = drop_duplicate_cols(pcp_df, aac_df)
centrality_df = drop_duplicate_cols(pd.concat([pcp_df, aac_df], axis=1), centrality_df)
go_df         = drop_duplicate_cols(pd.concat([pcp_df, aac_df, centrality_df], axis=1), go_df)

# ------------------------------------------------------------------
# 5. Sequential merge
#    - inner for core (PCP ∩ AAC) to keep only proteins in both sets
#    - left joins afterwards so we don't lose proteins that are
#      missing centrality or GO annotations
# ------------------------------------------------------------------
merged = pcp_df.merge(aac_df,        on="protein", how="inner")
merged = merged.merge(centrality_df, on="protein", how="left")
merged = merged.merge(go_df,         on="protein", how="left")

# ------------------------------------------------------------------
# 6. Save and quick sanity check
# ------------------------------------------------------------------
merged.to_csv(OUTPUT_FILE, index=False)
print(f"✅  Final feature matrix written to: {OUTPUT_FILE}  ({merged.shape[0]} proteins × {merged.shape[1]-1} features)")
merged.head()


## 🎉 Done

- **PCP & AAC** extracted via automated *pfeature* batch script.
- **Centrality metrics** computed in Cytoscape & imported.
- **GO one‑hot features** constructed for 61 curated terms.
- All blocks merged → `data/processed/merged_dataset.csv` (model‑ready).

Proceed to **`02_feature_selection.ipynb`** for downstream analysis.

## 5 Balanced Dataset Generation (47 Sets)

To address class imbalance between positive (SARS-CoV-2) and negative (human) proteins, we generate **47 balanced datasets** by:

- Fixing the 335 positive proteins
- Randomly sampling 335 negatives from a pool of ~15,000 reviewed human proteins
- Merging both sets and shuffling

Each dataset (`dataset1.csv`, ..., `dataset44.csv`) contains:
- 335 positive samples (label = 1)
- 335 negative samples (label = 0)

These balanced datasets are used for training 44 separate models for robust evaluation.


In [ ]:
import pandas as pd
import os

# Load positive and negative sets
positive = pd.read_csv('./data/processed/positive_final_feature_matrix.csv')
negative = pd.read_csv('./data/processed/negative_final_feature_matrix.csv')

# Check shape
assert positive.shape[0] == 335, "Positive dataset must have 335 proteins"
assert negative.shape[0] >= 335, "Not enough negatives to sample from"

# Add labels
positive['label'] = 1
negative['label'] = 0

# Output directory
output_dir = './data/processed/balanced_datasets/'
os.makedirs(output_dir, exist_ok=True)

# Generate 47 balanced datasets
for i in range(1, 48):  # 1 to 47
    sampled_neg = negative.sample(n=335, random_state=i)
    balanced = pd.concat([positive, sampled_neg]).sample(frac=1, random_state=i).reset_index(drop=True)
    balanced.to_csv(f'{output_dir}/dataset{i}.csv', index=False)
    print(f"✅ Created dataset{i}.csv with {balanced.shape[0]} samples")

print("✅ All 47 balanced datasets created in:", output_dir)
